In [2]:
import minsearch

In [3]:
import json

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [7]:
#Now we want to index it
index = minsearch.Index(
    text_fields=['question','text','section'],
    keyword_fields=['course']
)

In [8]:
q = 'the course has already started, can I still enroll?'

In [9]:
index.fit(documents)

In [10]:
boost = {'question': 3.0, 'section': 0.5}

result = index.search(
    query=q,
    boost_dict=boost,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    num_results=5
)

In [11]:
result

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [12]:
# from openai import OpenAI
from google import genai

In [13]:
# client = OpenAI()
client = genai.Client()

In [19]:
# response = client.chat.completions.create(
#     model='gpt-4o',
#     messages=[{"role": "user", "content": q}]
# )
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=q
)

In [21]:
print(response.text)

It **might be possible, but it depends heavily on several factors.**

Here's what you need to consider and what steps you should take:

**Factors that determine if you can still enroll:**

1.  **Institution's Late Registration Policy:**
    *   Most colleges and universities have a strict "add/drop" deadline, usually within the first week or two of the semester/quarter. If you're past this, it becomes much harder.
    *   Some online, self-paced, or vocational courses might be more flexible.

2.  **How long ago the course started:**
    *   **A few days ago:** Often manageable, especially if it's within the official add/drop period.
    *   **A week or two ago:** Possible, but you'll have a significant amount of catching up to do. You'll likely need special permission.
    *   **Several weeks or months ago:** Highly unlikely, as too much material, assignments, and exams would have been missed.

3.  **Type of Course:**
    *   **Lecture-based courses:** Might be easier to catch up in, a

In [22]:
#We want the LLM to put the document content in the context of the answer
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FQA database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
""".strip()

In [23]:
context = ""

for doc in result:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

In [24]:
prompt = prompt_template.format(question=q, context=context).strip()

In [26]:
# response = client.chat.completions.create(
#     model='gpt-4o',
#     messages=[{"role": "user", "content": prompt}]
# )

# response.choices[0].message.content
response = client.models.generate_content(
    model="gemini-2.5-flash",
    contents=prompt
)

print(response.text)

Yes, even if you don't register, you're still eligible to submit the homeworks. Be aware, however, that there will be deadlines for turning in the final projects.
